In [1]:
import sys
import os
import cv2
import numpy as np
import tensorflow
import keras
sys.path.append("..")

Using TensorFlow backend.


In [2]:
#Load images
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".ppm")]
        for f in file_names:
            images.append(cv2.imread(f))
            labels.append(int(d))
    return images, labels
ROOT_PATH = ""
train_data_directory = os.path.join(ROOT_PATH, "Training")
test_data_directory = os.path.join(ROOT_PATH, "Testing")


x_train, y_train = load_data(train_data_directory)
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test, y_test = load_data(test_data_directory)
x_test = np.array(x_test)
y_test = np.array(y_test)
#Preprocess
x_test = [cv2.resize(image, (28, 28)) for image in x_test]
x_train = [cv2.resize(image, (28, 28)) for image in x_train]
x_test = np.array(x_test)
x_train = np.array(x_train)

y_train = np.reshape(y_train,[y_train.shape[0],1])
y_test = np.reshape(y_test,[y_test.shape[0],1])

In [3]:
import matplotlib.pyplot as plt
plt.title('sample image')
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(x_train[i])

print("X_train shape:",x_train.shape)
img_shape = x_train[1].shape

('X_train shape:', (4575, 28, 28, 3))


In [31]:
NUM_CLASSES = 62
# convert class labels to one-hot encoded, should have shape (?, NUM_CLASSES)
y_train2 = keras.utils.to_categorical(y_train)
y_test2 = keras.utils.to_categorical(y_test)
# import necessary building blocks
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout,BatchNormalization,LSTM,Reshape
from keras.layers.advanced_activations import LeakyReLU

In [32]:
def make_model():    

    model = Sequential()
    model.add(Conv2D(input_shape=(28,28,3),padding="same",kernel_size=3,filters=16))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    
    model.add(Conv2D(padding="same",kernel_size=3,filters=32))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv2D(padding="same",kernel_size=3,filters=32))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    
    model.add(Conv2D(padding="same",kernel_size=3,filters=64))
    model.add(LeakyReLU(0.1))
    model.add(BatchNormalization())
    
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(Reshape((1, 256)))
    model.add(LSTM(256))
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES))
    model.add(LeakyReLU(0.1))
    
    model.add(Activation("softmax"))
    
    return model

In [33]:
model = make_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 28, 28, 16)        448       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 28, 28, 16)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 28, 28, 16)        64        
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 28, 28, 32)        4640      
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 32)        0         
__________

In [34]:
from keras import backend as K
INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 32
EPOCHS = 10


# don't call K.set_learning_phase() !!! (otherwise will enable dropout in train/test simultaneously)
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='categorical_crossentropy',  # we train 10-way classification
    optimizer=keras.optimizers.adamax(lr=INIT_LR),  # for SGD
    metrics=['accuracy']  # report accuracy during training
)

# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

# fit model
model.fit(
    x_train, y_train2,  # prepared data
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), LrHistory()],
    validation_data=(x_test, y_test2),
    shuffle=True,
    verbose=0
)

('Learning rate:', 0.005)
('Learning rate:', 0.0045)
('Learning rate:', 0.00405)
('Learning rate:', 0.003645)
('Learning rate:', 0.0032805)
('Learning rate:', 0.00295245)
('Learning rate:', 0.002657205)
('Learning rate:', 0.0023914846)
('Learning rate:', 0.002152336)
('Learning rate:', 0.0019371024)


In [35]:
test_predictions = model.predict_proba(x_test).argmax(axis=-1)

In [36]:
test_answers = y_test2.argmax(axis=-1)

In [37]:
test_accuracy = np.mean(test_predictions==test_answers)

In [38]:
print(test_accuracy*100,'%')

(89.28571428571429, '%')
